# Pandas Data Structures

In [36]:
# Creating DataFrame objects from files, API requests, SQL queries, and other Python objects
# Inspecting DataFrame objects and calculating summary statistics
# Grabbing subsets of the data via selection, slicing, indexing, and filtering

In [37]:
# files:  https://github.com/stefmolin/Hands-On-Data-Analysis-with-Pandas-2nd-edition/tree/master/ch_02
# US Geological Survey (USGS) 

In [38]:
# earthquakes.csv pulled from September 18, 2018 through October 13, 2018
# example_data.csv contains five rows and a subset of the columns from the earthquakes.csv
# tsunamis.csv file is a subset of the data in the earthquakes.csv
# quakes.db file contains a SQLite database with a single table for the tsunamis data

In [39]:
# NumPy provides the n-dimensional arrays that pandas builds upon - implemented as Python classes

In [40]:
import numpy as np

data = np.genfromtxt(
    'data/example_data.csv', delimiter=';', 
    names=True, dtype=None, encoding='UTF'
)
data

array([('2018-10-13 11:10:23.560', '262km NW of Ozernovskiy, Russia', 'mww', 6.7, 'green', 1),
       ('2018-10-13 04:34:15.580', '25km E of Bitung, Indonesia', 'mww', 5.2, 'green', 0),
       ('2018-10-13 00:13:46.220', '42km WNW of Sola, Vanuatu', 'mww', 5.7, 'green', 0),
       ('2018-10-12 21:09:49.240', '13km E of Nueva Concepcion, Guatemala', 'mww', 5.7, 'green', 0),
       ('2018-10-12 02:52:03.620', '128km SE of Kimbe, Papua New Guinea', 'mww', 5.6, 'green', 1)],
      dtype=[('time', '<U23'), ('place', '<U37'), ('magType', '<U3'), ('mag', '<f8'), ('alert', '<U5'), ('tsunami', '<i4')])

In [41]:
#  information about the dimensions of the array and the data types 
data.shape

(5,)

In [42]:
data.dtype

dtype([('time', '<U23'), ('place', '<U37'), ('magType', '<U3'), ('mag', '<f8'), ('alert', '<U5'), ('tsunami', '<i4')])

In [45]:
#  NumPy arrays contain a single data type

In [44]:
# select the third index of each row
# to see how long this implementation takes:
%%timeit
max([row[3] for row in data])

5.53 µs ± 244 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [46]:
# dictionary comprehension: keys are the column names and the values are NumPy arrays of the data
# early twice as fast as the list comprehension implementation,
array_dict = {
    col: np.array([row[i] for row in data])
    for i, col in enumerate(data.dtype.names)
}
array_dict

{'time': array(['2018-10-13 11:10:23.560', '2018-10-13 04:34:15.580',
        '2018-10-13 00:13:46.220', '2018-10-12 21:09:49.240',
        '2018-10-12 02:52:03.620'], dtype='<U23'),
 'place': array(['262km NW of Ozernovskiy, Russia', '25km E of Bitung, Indonesia',
        '42km WNW of Sola, Vanuatu',
        '13km E of Nueva Concepcion, Guatemala',
        '128km SE of Kimbe, Papua New Guinea'], dtype='<U37'),
 'magType': array(['mww', 'mww', 'mww', 'mww', 'mww'], dtype='<U3'),
 'mag': array([6.7, 5.2, 5.7, 5.7, 5.6]),
 'alert': array(['green', 'green', 'green', 'green', 'green'], dtype='<U5'),
 'tsunami': array([1, 0, 0, 0, 1])}

In [47]:
%%timeit
array_dict['mag'].max()

2.74 µs ± 138 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [48]:
#a NumPy array of strings
np.array([
    value[array_dict['mag'].argmax()] 
    for key, value in array_dict.items()
])

array(['2018-10-13 11:10:23.560', '262km NW of Ozernovskiy, Russia',
       'mww', '6.7', 'green', '1'], dtype='<U31')

## Series data structure

In [49]:
import pandas as pd

place = pd.Series(array_dict['place'], name='place')
place
# data type of place is object

0          262km NW of Ozernovskiy, Russia
1              25km E of Bitung, Indonesia
2                42km WNW of Sola, Vanuatu
3    13km E of Nueva Concepcion, Guatemala
4      128km SE of Kimbe, Papua New Guinea
Name: place, dtype: object

In [50]:
# https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.html

### Index class

In [51]:
# row labels enable selection by row
place_index = place.index
place_index

RangeIndex(start=0, stop=5, step=1)

In [54]:
# Index object is built on top of a NumPy array
place_index.values

array([0, 1, 2, 3, 4], dtype=int64)

In [56]:
# arithmetic operations - element-wise
np.array([1, 1, 1]) + np.array([-1, 0, 1])

array([0, 1, 2])

### DataFrame class

In [58]:
# builds upon the Series class and can have many columns, each with its own data type
df = pd.DataFrame(array_dict)
df

,time,place,magType,mag,alert,tsunami
0,2018-10-13 11:10:23.560,"262km NW of Ozernovskiy, Russia",mww,6.7,green,1
1,2018-10-13 04:34:15.580,"25km E of Bitung, Indonesia",mww,5.2,green,0
2,2018-10-13 00:13:46.220,"42km WNW of Sola, Vanuatu",mww,5.7,green,0
3,2018-10-12 21:09:49.240,"13km E of Nueva Concepcion, Guatemala",mww,5.7,green,0
4,2018-10-12 02:52:03.620,"128km SE of Kimbe, Papua New Guinea",mww,5.6,green,1


In [59]:
df.dtypes

time        object
place       object
magType     object
mag        float64
alert       object
tsunami      int32
dtype: object

In [60]:
df.values

array([['2018-10-13 11:10:23.560', '262km NW of Ozernovskiy, Russia',
        'mww', 6.7, 'green', 1],
       ['2018-10-13 04:34:15.580', '25km E of Bitung, Indonesia', 'mww',
        5.2, 'green', 0],
       ['2018-10-13 00:13:46.220', '42km WNW of Sola, Vanuatu', 'mww',
        5.7, 'green', 0],
       ['2018-10-12 21:09:49.240',
        '13km E of Nueva Concepcion, Guatemala', 'mww', 5.7, 'green', 0],
       ['2018-10-12 02:52:03.620', '128km SE of Kimbe, Papua New Guinea',
        'mww', 5.6, 'green', 1]], dtype=object)

In [61]:
df.columns

Index(['time', 'place', 'magType', 'mag', 'alert', 'tsunami'], dtype='object')

In [62]:
# Pandas will only perform the operation when both the index and column match
df + df

,time,place,magType,mag,alert,tsunami
0,2018-10-13 11:10:23.5602018-10-13 11:10:23.560,"262km NW of Ozernovskiy, Russia262km NW of Oze...",mwwmww,13.4,greengreen,2
1,2018-10-13 04:34:15.5802018-10-13 04:34:15.580,"25km E of Bitung, Indonesia25km E of Bitung, I...",mwwmww,10.4,greengreen,0
2,2018-10-13 00:13:46.2202018-10-13 00:13:46.220,"42km WNW of Sola, Vanuatu42km WNW of Sola, Van...",mwwmww,11.4,greengreen,0
3,2018-10-12 21:09:49.2402018-10-12 21:09:49.240,"13km E of Nueva Concepcion, Guatemala13km E of...",mwwmww,11.4,greengreen,0
4,2018-10-12 02:52:03.6202018-10-12 02:52:03.620,"128km SE of Kimbe, Papua New Guinea128km SE of...",mwwmww,11.2,greengreen,2


## Creating a pandas DataFrame

In [63]:
import datetime as dt
import numpy as np
import pandas as pd

In [64]:
# how to make a Series object (series of five random numbers between 0 and 1)
# seed gives a starting point for the generation of pseudorandom numbers

np.random.seed(0) # set a seed for reproducibility
pd.Series(np.random.rand(5), name='random')

0    0.548814
1    0.715189
2    0.602763
3    0.544883
4    0.423655
Name: random, dtype: float64

In [65]:
# Creating a DataFrame object from a Series object
pd.Series(np.linspace(0, 10, num=5)).to_frame()

,0
0,0.0
1,2.5
2,5.0
3,7.5
4,10.0


In [66]:
# Creating a DataFrame from Python Data Structures
# to turn a single Series object into a DataFrame object: use its to_frame() method
# columns can all be different data types
# package the columns in a dictionary

np.random.seed(0) 
pd.DataFrame(
    {
        'random': np.random.rand(5),
        'text': ['hot', 'warm', 'cool', 'cold', None],
        'truth': [np.random.choice([True, False]) for _ in range(5)]
    }, 
    index=pd.date_range(
        end=dt.date(2019, 4, 21),
        freq='1D',
        periods=5, 
        name='date'
    )
)

# By convention, we use _ to hold variables in a loop that we don't care about
# https://hackernoon.com/understanding-the-underscore-of-python-309d1a029edc

,random,text,truth
date,,,
2019-04-17,0.548814,hot,False
2019-04-18,0.715189,warm,True
2019-04-19,0.602763,cool,True
2019-04-20,0.544883,cold,False
2019-04-21,0.423655,None,True


In [67]:
pd.DataFrame([
    {'mag': 5.2, 'place': 'California'},
    {'mag': 1.2, 'place': 'Alaska'},
    {'mag': 0.2, 'place': 'California'},
])

,mag,place
0,5.2,California
1,1.2,Alaska
2,0.2,California


In [68]:
list_of_tuples = [(n, n**2, n**3) for n in range(5)]
list_of_tuples

[(0, 0, 0), (1, 1, 1), (2, 4, 8), (3, 9, 27), (4, 16, 64)]

In [69]:
pd.DataFrame(
    list_of_tuples, 
    columns=['n', 'n_squared', 'n_cubed']
)

,n,n_squared,n_cubed
0,0,0,0
1,1,1,1
2,2,4,8
3,3,9,27
4,4,16,64


In [70]:
pd.DataFrame(
    np.array([
        [0, 0, 0],
        [1, 1, 1],
        [2, 4, 8],
        [3, 9, 27],
        [4, 16, 64]
    ]), columns=['n', 'n_squared', 'n_cubed']
)

,n,n_squared,n_cubed
0,0,0,0
1,1,1,1
2,2,4,8
3,3,9,27
4,4,16,64


## Creating a DataFrame object from the contents of a CSV File